In [3]:
from itertools import combinations, product
from mip.model import *
from pycosat import solve
from shapely.geometry import LineString

# Queens

In [4]:
n = 20
clauses = []


# convert pairs of integers to a unique integer
def varnum(i, j):
    assert i in range(n) and j in range(n)
    return i * n + j + 1


# each row contains at least one queen
for i in range(n):
    clauses.append([varnum(i, j) for j in range(n)])

# each row contains at most one queen
for i in range(n):
    for j1, j2 in combinations(range(n), 2):
        clauses.append([-varnum(i, j1), -varnum(i, j2)])

# each column contains at most one queen
for j in range(n):
    for i1, i2 in combinations(range(n), 2):
        clauses.append([-varnum(i1, j), -varnum(i2, j)])

# no two queens stay on the same diagonal
for i1, j1, i2, j2 in product(range(n), repeat=4):
    if i1 == i2:
        continue

    if abs(i1 - i2) == abs(j1 - j2):
        clauses.append([-varnum(i1, j1),
                        -varnum(i2, j2)])


assignment = solve(clauses)
for i, j in product(range(n), repeat=2):
    if assignment[varnum(i, j) - 1] > 0:
        print(j, end=' ')

15 18 9 17 5 16 1 11 8 3 0 2 14 12 19 7 13 10 6 4 

# Diagonals

In [5]:
n = 5

model = Model()
model.verbose = False

segments = {}
for i, j in product(range(n + 1), repeat=2):
    for delta_i, delta_j in product((-1, 1), repeat=2):
        if i + delta_i in range(n + 1) and j + delta_j in range(n + 1):
            segments[i, j, i + delta_i, j + delta_j] = \
                model.add_var(var_type=BINARY)

for s1, s2 in combinations(segments, 2):
    if LineString([s1[:2], s1[2:]]).intersects(LineString([s2[:2], s2[2:]])):
        model += segments[s1] + segments[s2] <= 1

model.objective = maximize(xsum(segments.values()))
model.optimize()

print(model.objective_value)
for s in segments:
    if abs(segments[s].x) > 1e-6:
        print(s)

gurobi version 8.1 found


16.0
(0, 2, 1, 3)
(1, 0, 0, 1)
(1, 1, 2, 0)
(1, 2, 2, 3)
(1, 4, 0, 3)
(1, 5, 0, 4)
(2, 4, 3, 3)
(2, 5, 3, 4)
(3, 0, 2, 1)
(3, 1, 2, 2)
(4, 1, 5, 2)
(4, 2, 5, 3)
(4, 3, 3, 2)
(4, 4, 3, 5)
(5, 1, 4, 0)
(5, 4, 4, 5)
